__Import dependencies and decleare global variables__

In [17]:
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import os
import re
import random
from keras.models import Model
from sklearn.utils import shuffle
from keras.layers import Dense, LSTM, CuDNNLSTM, Input, Embedding, TimeDistributed, Flatten, Dropout
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from copy import deepcopy
from keras.models import load_model

EPOCHS = 1
MAX_LENGTH = 30
BATCH_SIZE = 64
EMBEDDING_DIM = 128
HIDDEN_DIM = 1024
NUM_PLOTS = 1000

__Import data__

In [18]:
def clean(text):
    '''
    '''
    text = text.strip()
    text = text.replace("ain't", "am not")
    text = text.replace("aren't", "are not")
    text = text.replace("can't", "cannot")
    text = text.replace("can't've", "cannot have")
    text = text.replace("'cause", "because")
    text = text.replace("could've", "could have")
    text = text.replace("couldn't", "could not")
    text = text.replace("couldn't've", "could not have")
    text = text.replace("should've", "should have")
    text = text.replace("should't", "should not")
    text = text.replace("should't've", "should not have")
    text = text.replace("would've", "would have")
    text = text.replace("would't", "would not")
    text = text.replace("would't've", "would not have")
    text = text.replace("didn't", "did not")
    text = text.replace("doesn't", "does not")
    text = text.replace("don't", "do not")
    text = text.replace("hadn't", "had not")
    text = text.replace("hadn't've", "had not have")
    text = text.replace("hasn't", "has not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd", "he would")
    text = text.replace("haven't", "have not")
    text = text.replace("he'd've", "he would have")
    text = text.replace("'s", "")
    text = text.replace("'t", "")
    text = text.replace("'ve", "")
    text = text.replace(".", " . ")
    text = text.replace("!", " ! ")
    text = text.replace("?", " ? ")
    text = text.replace(";", " ; ")
    text = text.replace(":", " : ")
    text = text.replace("\'", "")
    text = text.replace("\"", "")
    text = text.replace(",", "")
    text = text.replace("[", "")
    text = text.replace("]","")
    text = text.replace("{","")
    text = text.replace("}", "")
    text = text.replace("/", "")
    text = text.replace("|", "")
    text = text.replace("-", "")
    text = text.replace("(", "")
    text = text.replace(")", "")
    text = text.replace("$", "")
    text = text.replace("+", "")
    text = text.replace("*", "")
    text = text.replace("%", "")
    text = text.replace("#", "")
    text = text.lower()
    text = ''.join([i for i in text if not i.isdigit()])

    return text

try:
    
    dirname = os.path.abspath('')
    filepath = os.path.join(dirname, 'input_data/wiki_movie_plots.csv')
    dataframe = pd.read_csv(filepath, sep=',')
    plotsList = dataframe['Plot']
    plotsList = plotsList[:NUM_PLOTS]
    print('{} plots imported.'.format(len(plotsList)))
    for idx, p in enumerate(plotsList):
        plotsList[idx] = clean(p)
    print('{} plots cleaned.'.format(len(plotsList)))
except IOError:
    
    sys.exit('Cannot find data!')

1000 plots imported.
1000 plots cleaned.


__Extract Vocabulary__

In [19]:
# CREATE VOCABULARY OF WORDS
idx2word = []
word2idx = {'<PAD>' : 0, '<START>' : 1 , '<END>': 2}
wordSequence = []
for plot in plotsList:
    words = plot.split(' ')
    wordSequence.extend(words)
    for word in words:
        if word not in word2idx:
            word2idx[word] = len(word2idx)

for word in idx2word:
    word2idx[word] = len(word2idx)

idx2word = list(word2idx.keys())
textAsInt = np.array([word2idx[w] for w in wordSequence])
vocab_size = len(idx2word)
print('Vocabulary Size: {}'.format(vocab_size))


Vocabulary Size: 17190


__Preprocess__

In [20]:
inputSentences = []
targetSentences = []
outputSentences = []

for plot in plotsList:
        words = plot.split(' ')

        b=True
        while b:
            if('' in words): 
                words.remove('')
            else: b = False

        sentences = [words[i:i+MAX_LENGTH] for i in range(0, len(words), MAX_LENGTH)]
        for s in sentences:
            for i in range(1, len(s)):
                encode_tokens, decode_tokens = s[:i], s[i:]
                encode_tokens = ' '.join(['<START>'] + encode_tokens + ['<END>'])
                output_tokens = ' '.join(decode_tokens + ['<END>'])
                decode_tokens = ' '.join(['<START>'] + decode_tokens + ['<END>'])
                inputSentences.append(encode_tokens)
                targetSentences.append(decode_tokens)
                outputSentences.append(output_tokens)

numSamples = len(inputSentences)
print('Num samples: {}'.format(numSamples))

print("Creating dataset to feed Model . . . ")
dirname = os.path.abspath('')
filePath = os.path.join(dirname, os.path.join(dirname, 'preprocessed/dataset_ed_plots_{}_{}.csv'.format(
MAX_LENGTH,  
NUM_PLOTS)))

if os.path.exists(filePath):
    os.remove(filePath) 

d= {'input_encoder' : inputSentences, 'input_decoder' :targetSentences, 'output_decoder':outputSentences }
df = pd.DataFrame(data=d) 
df = shuffle(df)
df.to_csv(filePath, index=False)

print("Dataset printed on CSV.")

Num samples: 224706
Creating dataset to feed Model . . . 
Dataset printed on CSV.


In [21]:
def generate_data(word_2_idx, num_samples, max_length, vocab_length, batch_size=BATCH_SIZE):
    '''
    '''
    dirname = os.path.abspath('')
    filePath = os.path.join(dirname, os.path.join(dirname, 'preprocessed/dataset_ed_plots_{}_{}.csv'.format(
    MAX_LENGTH,  
    NUM_PLOTS)))
    df = pd.read_csv(filePath)
    
    encoderInputData = np.zeros((numSamples, max_length + 2), dtype='int')
    decoderInputData = np.zeros((numSamples, max_length + 2), dtype='int')
    decoderTargetData = np.zeros((numSamples, max_length + 2, 1),dtype='int')
    
    for i in range(0, numSamples):
        print('Generating... {}/{}'.format(i,numSamples))
        encoderTokens = df.iloc[[i]]['input_encoder'].values[0].split(' ')
        decoderTokens = df.iloc[[i]]['input_decoder'].values[0].split(' ')
        outputTokens = df.iloc[[i]]['output_decoder'].values[0].split(' ')

        for t, word in enumerate(encoderTokens):
            encoderInputData[i, t] = word_2_idx[word]
        for t, word in enumerate(decoderTokens):
            decoderInputData[i, t] = word_2_idx[word]
        for t, word in enumerate(outputTokens):
            # decoderTargetData is ahead of decoderInputData by one timestep
            decoderTargetData[i, t, 0] = word_2_idx[word]

    
    return encoderInputData, decoderInputData, decoderTargetData

__Define function to build the model__

In [22]:
def build_encoder(vocab_length, embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM):
    '''
    '''
    # Define an input sequence and process it.
    # Input layer of the encoder :
    encoderInput = Input(shape=(None,))
    
    # Hidden layers of the encoder :
    encoder_embedding = Embedding(input_dim = vocab_length, output_dim = embedding_dim)(encoderInput)

    # Output layer of the encoder :
    encoder_LSTM = CuDNNLSTM(hidden_dim , return_state=True)
    encoder_outputs, state_h, state_c = encoder_LSTM(encoder_embedding)

    # We discard `encoder_outputs` and only keep the states.
    encoderStates = [state_h, state_c]
    
    
    return encoderInput, encoderStates


def build_encoder_gen(encoder_input, encoder_states):
    '''
    '''
    encoderModelGen = Model(encoder_input, encoder_states)

    return encoderModelGen


def build_decoder(vocab_length, encoderStates, embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM):
    '''
    '''
    # Set up the decoder, using `encoderStates` as initial state.
    # Input layer of the decoder :
    decoderInput = Input(shape=(None,))

    # Hidden layers of the decoder :
    decoderEmbeddingLayer = Embedding(input_dim = vocab_length, output_dim = embedding_dim)
    decoder_embedding = decoderEmbeddingLayer(decoderInput)

    decoderLSTMLayer = CuDNNLSTM(hidden_dim , return_sequences=True, return_state=True)
    decoder_LSTM_output, _ , _ = decoderLSTMLayer(decoder_embedding, initial_state=encoderStates)

    # Output layer of the decoder :
    decoderDenseLayer = Dense(vocab_length, activation='softmax')
    decoderOutput = decoderDenseLayer(decoder_LSTM_output)

    return decoderInput, decoderOutput, decoderEmbeddingLayer,  decoderLSTMLayer, decoderDenseLayer


def build_decoder_gen(decoder_input, decoder_embedding_layer, decoder_LSTM_layer, decoder_dense, hidden_dim=HIDDEN_DIM):
    '''
    '''
    decoder_state_input_h = Input(shape=(hidden_dim,))
    decoder_state_input_c = Input(shape=(hidden_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

    decoder_embedding_gen = decoder_embedding_layer(decoder_input)
    decoder_LSTM_output_gen, state_h_gen , state_c_gen = decoder_LSTM_layer(decoder_embedding_gen, initial_state = decoder_states_inputs)
    decoder_states_gen = [state_h_gen, state_c_gen]
    decoderOutputGen = decoder_dense(decoder_LSTM_output_gen)

    # sampling model will take encoder states and decoder_input(seed initially) and output the predictions(french word index) We dont care about decoder_states2
    decoderModelGen = Model(
    [decoder_input] + decoder_states_inputs,
    [decoderOutputGen] + decoder_states_gen
    )

    return decoderModelGen
  
def build_encoder_decoder_model(encoder_input, decoder_input, decoder_output):
    '''
    '''
    model = Model([encoder_input, decoder_input], decoder_output)
    model.summary()

    return model

__Train model__

In [23]:
dirname = os.path.abspath('')

encoderGenPath = os.path.join(dirname, 'models/encoder_plots_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_PLOTS)
)

decoderGenPath = os.path.join(dirname, 'models/decoder_plots_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_PLOTS)
)

encoderInput, encoderStates = build_encoder(vocab_length=vocab_size)

decoderInput, decoderOutput, decoderEmbeddingLayer,  decoderLSTMLayer, decoderDenseLayer = build_decoder(
    vocab_length=vocab_size, 
    encoderStates=encoderStates
)

model = build_encoder_decoder_model(
    encoder_input=encoderInput, 
    decoder_input=decoderInput, 
    decoder_output=decoderOutput
)

encoderInputData, decoderInputData, decoderTargetData = generate_data(
    word_2_idx=word2idx,
    num_samples=numSamples,
    max_length=MAX_LENGTH, 
    vocab_length=vocab_size
)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoderInputData, decoderInputData], decoderTargetData, batch_size=BATCH_SIZE, epochs=EPOCHS)

encoderModelGen = build_encoder_gen(
    encoder_input = encoderInput, 
    encoder_states = encoderStates
)

decoderModelGen = build_decoder_gen(
    decoder_input = decoderInput, 
    decoder_embedding_layer = decoderEmbeddingLayer, 
    decoder_LSTM_layer = decoderLSTMLayer, 
    decoder_dense = decoderDenseLayer
)

encoderModelGen.save_model(encoderGenPath)
decoderModelGen.save_model(decoderGenPath)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, None, 128)    2200320     input_7[0][0]                    
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, None, 128)    2200320     input_8[0][0]                    
__________________________________________________________________________________________________
cu_dnnlstm

Generating... 485/224706
Generating... 486/224706
Generating... 487/224706
Generating... 488/224706
Generating... 489/224706
Generating... 490/224706
Generating... 491/224706
Generating... 492/224706
Generating... 493/224706
Generating... 494/224706
Generating... 495/224706
Generating... 496/224706
Generating... 497/224706
Generating... 498/224706
Generating... 499/224706
Generating... 500/224706
Generating... 501/224706
Generating... 502/224706
Generating... 503/224706
Generating... 504/224706
Generating... 505/224706
Generating... 506/224706
Generating... 507/224706
Generating... 508/224706
Generating... 509/224706
Generating... 510/224706
Generating... 511/224706
Generating... 512/224706
Generating... 513/224706
Generating... 514/224706
Generating... 515/224706
Generating... 516/224706
Generating... 517/224706
Generating... 518/224706
Generating... 519/224706
Generating... 520/224706
Generating... 521/224706
Generating... 522/224706
Generating... 523/224706
Generating... 524/224706


Generating... 974/224706
Generating... 975/224706
Generating... 976/224706
Generating... 977/224706
Generating... 978/224706
Generating... 979/224706
Generating... 980/224706
Generating... 981/224706
Generating... 982/224706
Generating... 983/224706
Generating... 984/224706
Generating... 985/224706
Generating... 986/224706
Generating... 987/224706
Generating... 988/224706
Generating... 989/224706
Generating... 990/224706
Generating... 991/224706
Generating... 992/224706
Generating... 993/224706
Generating... 994/224706
Generating... 995/224706
Generating... 996/224706
Generating... 997/224706
Generating... 998/224706
Generating... 999/224706
Generating... 1000/224706
Generating... 1001/224706
Generating... 1002/224706
Generating... 1003/224706
Generating... 1004/224706
Generating... 1005/224706
Generating... 1006/224706
Generating... 1007/224706
Generating... 1008/224706
Generating... 1009/224706
Generating... 1010/224706
Generating... 1011/224706
Generating... 1012/224706
Generating..

Generating... 1566/224706
Generating... 1567/224706
Generating... 1568/224706
Generating... 1569/224706
Generating... 1570/224706
Generating... 1571/224706
Generating... 1572/224706
Generating... 1573/224706
Generating... 1574/224706
Generating... 1575/224706
Generating... 1576/224706
Generating... 1577/224706
Generating... 1578/224706
Generating... 1579/224706
Generating... 1580/224706
Generating... 1581/224706
Generating... 1582/224706
Generating... 1583/224706
Generating... 1584/224706
Generating... 1585/224706
Generating... 1586/224706
Generating... 1587/224706
Generating... 1588/224706
Generating... 1589/224706
Generating... 1590/224706
Generating... 1591/224706
Generating... 1592/224706
Generating... 1593/224706
Generating... 1594/224706
Generating... 1595/224706
Generating... 1596/224706
Generating... 1597/224706
Generating... 1598/224706
Generating... 1599/224706
Generating... 1600/224706
Generating... 1601/224706
Generating... 1602/224706
Generating... 1603/224706
Generating..

Generating... 2078/224706
Generating... 2079/224706
Generating... 2080/224706
Generating... 2081/224706
Generating... 2082/224706
Generating... 2083/224706
Generating... 2084/224706
Generating... 2085/224706
Generating... 2086/224706
Generating... 2087/224706
Generating... 2088/224706
Generating... 2089/224706
Generating... 2090/224706
Generating... 2091/224706
Generating... 2092/224706
Generating... 2093/224706
Generating... 2094/224706
Generating... 2095/224706
Generating... 2096/224706
Generating... 2097/224706
Generating... 2098/224706
Generating... 2099/224706
Generating... 2100/224706
Generating... 2101/224706
Generating... 2102/224706
Generating... 2103/224706
Generating... 2104/224706
Generating... 2105/224706
Generating... 2106/224706
Generating... 2107/224706
Generating... 2108/224706
Generating... 2109/224706
Generating... 2110/224706
Generating... 2111/224706
Generating... 2112/224706
Generating... 2113/224706
Generating... 2114/224706
Generating... 2115/224706
Generating..

...

Generating... 27546/224706
Generating... 27547/224706
Generating... 27548/224706
Generating... 27549/224706
Generating... 27550/224706
Generating... 27551/224706
Generating... 27552/224706
Generating... 27553/224706
Generating... 27554/224706
Generating... 27555/224706
Generating... 27556/224706
Generating... 27557/224706
Generating... 27558/224706
Generating... 27559/224706
Generating... 27560/224706
Generating... 27561/224706
Generating... 27562/224706
Generating... 27563/224706
Generating... 27564/224706
Generating... 27565/224706
Generating... 27566/224706
Generating... 27567/224706
Generating... 27568/224706
Generating... 27569/224706
Generating... 27570/224706
Generating... 27571/224706
Generating... 27572/224706
Generating... 27573/224706
Generating... 27574/224706
Generating... 27575/224706
Generating... 27576/224706
Generating... 27577/224706
Generating... 27578/224706
Generating... 27579/224706
Generating... 27580/224706
Generating... 27581/224706
Generating... 27582/224706
G

Generating... 28034/224706
Generating... 28035/224706
Generating... 28036/224706
Generating... 28037/224706
Generating... 28038/224706
Generating... 28039/224706
Generating... 28040/224706
Generating... 28041/224706
Generating... 28042/224706
Generating... 28043/224706
Generating... 28044/224706
Generating... 28045/224706
Generating... 28046/224706
Generating... 28047/224706
Generating... 28048/224706
Generating... 28049/224706
Generating... 28050/224706
Generating... 28051/224706
Generating... 28052/224706
Generating... 28053/224706
Generating... 28054/224706
Generating... 28055/224706
Generating... 28056/224706
Generating... 28057/224706
Generating... 28058/224706
Generating... 28059/224706
Generating... 28060/224706
Generating... 28061/224706
Generating... 28062/224706
Generating... 28063/224706
Generating... 28064/224706
Generating... 28065/224706
Generating... 28066/224706
Generating... 28067/224706
Generating... 28068/224706
Generating... 28069/224706
Generating... 28070/224706
G

Generating... 28566/224706
Generating... 28567/224706
Generating... 28568/224706
Generating... 28569/224706
Generating... 28570/224706
Generating... 28571/224706
Generating... 28572/224706
Generating... 28573/224706
Generating... 28574/224706
Generating... 28575/224706
Generating... 28576/224706
Generating... 28577/224706
Generating... 28578/224706
Generating... 28579/224706
Generating... 28580/224706
Generating... 28581/224706
Generating... 28582/224706
Generating... 28583/224706
Generating... 28584/224706
Generating... 28585/224706
Generating... 28586/224706
Generating... 28587/224706
Generating... 28588/224706
Generating... 28589/224706
Generating... 28590/224706
Generating... 28591/224706
Generating... 28592/224706
Generating... 28593/224706
Generating... 28594/224706
Generating... 28595/224706
Generating... 28596/224706
Generating... 28597/224706
Generating... 28598/224706
Generating... 28599/224706
Generating... 28600/224706
Generating... 28601/224706
Generating... 28602/224706
G

Generating... 29338/224706
Generating... 29339/224706
Generating... 29340/224706
Generating... 29341/224706
Generating... 29342/224706
Generating... 29343/224706
Generating... 29344/224706
Generating... 29345/224706
Generating... 29346/224706
Generating... 29347/224706
Generating... 29348/224706


KeyboardInterrupt: 

__Generate text__

In [ ]:
def generate_text(sentences, encoder_model, decoder_model, vocab_length, word_2_idx, idx_2_word, max_length):
    '''
    '''
    for phrase in sentences:

        # Cleaning sentence
        phrase = clean(phrase)
        print('GENEREATING FROM: {}'.format(phrase))
        tokens = phrase.split(' ')
        inputSequence = np.zeros((1, max_length), dtype='int')
        for i, t in enumerate(tokens):
            inputSequence[0, i] = word_2_idx[t]

        # Encode the input as state vectors.
        statesValue = encoder_model.predict(inputSequence)
        # Generate empty target sequence of length 1.
        targetSeq = np.zeros((1, 1))
        targetSeq[0, 0] = word_2_idx['<START>']
        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stopCondition = False
        decodedSentence = ''
        decodedList = []
        while not stopCondition:
            outputTokens, h, c = decoder_model.predict(
                [targetSeq] + statesValue)

            # Sample a token
            print(outputTokens)
            sampledTokenIndex = np.argmax(outputTokens[0, -1, :])
            sampledWord = idx_2_word[sampledTokenIndex]
            decodedList.append(sampledWord)
            decodedSentence += ' ' + sampledWord
            print(decodedSentence)

            # Exit condition: either hit max length
            # or find stop character.
            if (sampledWord == '<END>' or len(decodedList)== max_length):
                stopCondition = True

            # Update the target sequence (of length 1).
            targetSeq = np.zeros((1, 1))
            targetSeq[0, 0] = sampledTokenIndex

            # Update states
            statesValue = [h, c]

        print('GENERATED: {}'.format(decodedSentence))

dirname = os.path.abspath('')

encoderGenPath = os.path.join(dirname, 'models/encoder_plots_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_PLOTS)
)

decoderGenPath = os.path.join(dirname, 'models/decoder_plots_{}_{}_{}_{}_{}_{}.h5'.format(
    EPOCHS, 
    MAX_LENGTH, 
    BATCH_SIZE, 
    EMBEDDING_DIM,
    HIDDEN_DIM,
    NUM_PLOTS)
)

encoderModel = load_model(encoderGenPath)
decoderModel = load_model(decoderGenPath)
generate_text(
    sentences = ['On a beautiful summer day three people', 'One day three people decide to take a trip' , 'The president is in trouble'],
    encoder_model = encoderModel,
    decoder_model = decoderModel, 
    vocab_length = vocabLength, 
    word_2_idx = word2idx, 
    idx_2_word = idx2word, 
    max_length = maxLength
)